# mwb_flow.prep Package Example 1.1
This example is update to the prep_Example1_PullData.ipynd and serves three purposes. First, it shows a workflow for creating a dataset of monthly precip and temperature data that can be used as an input for the mwb_flow model. Second, it compares three methods of retriving GridMet data and calculating zonal stats for each polygon. Lastly, it contains code for each proposed method that can be refrenced if needed later.  

In [1]:
# More imports than nessasary...
import os
from pathlib import Path
import geopandas as gpd
import xarray as xr

# Block of imports needed for GRIDtool.grid_area_weighted_volume
import rasterio as rio
import pandas as pd
from chmdata.thredds import GridMet, BBox
import numpy as np
from shapely.geometry import Polygon



mwb_flow_dir = r'C:\Users\CND905\Downloaded_Programs\mwb_flow'
os.chdir(mwb_flow_dir)

from prep.datafile import CreateInputFile
from prep.metdata import get_gridmet_at_points, get_gridmet_for_polygons
from prep.datafile import check_format

import py3dep
from tqdm import tqdm
from prep.utils import get_gridmet_cells
from config import GRIDMET_PARAMS
import GRIDtools as gt
import rioxarray
import xvec 
import shapely
from geocube.api.core import make_geocube



Initializing mwb_flow.prep module.


## Load in Shape File
Import a shape file with deliniated watershed polygon. This shape file has an attribute table with a column used to index the geometries. In this case, a column with the gage station numbers was used. 

In [2]:
exres_pth = Path(r'C:\Users\CND905\Downloaded_Programs\mwb_flow\Examples\data\Lolo_WB_Model_Calibration_Catchments_32611.shp')
exres = gpd.read_file(exres_pth)
exres = exres.to_crs(4326) # This file is in crs 32611 (WGS84 UTM zone 11N), need it to be 4326 for getting GridMET.

## Pull GridMet Data and Perform Zonal Stats
Three functions from prep.metdata.py are shown below. The first two functions, 'get_gridmet_for_polygons()' and 'get_gridmet_for_polygons_xvec()' pull meterologic data and perform area zonal stats (area weighted for precip and mean for temperature parameters) for polygons but use diffrent packages to do so. More details for each function below.

The last function 'get_gridmet_at_points()' shows the original function that pulls meterologic data but only calculates the mean for polygons. The function is slow. 

### get_gridmet_for_polygons()

This function uses the geocube package which rasterizes all of the polygons at once. <ins>This function appears to be the fastest so we will use it for the rest of the workflow.</ins>

This range of dates was used since there are no data-gaps in the discharge record for each location. 

In [ ]:
exres_met = get_gridmet_for_polygons(exres, "gageID", start='2016-01-01', end='2019-12-31')
exres_met

## Format Dataset to Use with MWB_Flow model

Calculate the monthly average temperature.

In [ ]:
# Calculate daily mean air temperature then followed by the monthly mean air temperature
mean_temp = ((exres_met.min_temp + exres_met.max_temp) / 2) - 273.15  # also convert to Celcius from GridMET native Kelvin
mean_temp
monthly_temp = mean_temp.resample(time = "MS").mean()

# Convert to a DataArray with attributes and title
Monthly_Temp = xr.DataArray(monthly_temp, coords=monthly_temp.coords, attrs={'standard_name': 'Monthly_Temperature', 'units': 'Celcius'})
Monthly_Temp.name = 'mo_temp'
Monthly_Temp

Calculate the monthly precipitation.

In [ ]:
# Calculate the monthly precipitation in mm
monthly_precip = exres_met.precip_volume.resample(time = "MS").sum() # summarize by monthly sum
area_m2 = (exres_met['area'].values * 1000000) # convert polygon areas from km^2 to m^2
monthly_precip = (monthly_precip / np.tile(area_m2, (len(monthly_precip['time']), 1))) * 1000 # convert precip volume (m^3) to length (m) and then to mm

# Convert to a DataArray with attributes and title
Monthly_Precip = xr.DataArray(monthly_precip, coords=monthly_precip.coords, attrs={'standard_name': 'Monthy Precipitation', 'units': 'mm'})
Monthly_Precip.name = 'mo_precip'
Monthly_Precip

Load in discharge data downloaded and formatted from the q_datafile.py script. 

This discharge data was loaded in as a monthly volume in m^3 then converted to mm.

In [ ]:
monthly_q = xr.load_dataarray(r'C:\Users\CND905\Downloaded_Programs\mwb_flow\prep\q_datafile_output.nc')
monthly_q


# This is the master key with areas of each location
area_df = pd.DataFrame(exres_met['area'].values).set_index(exres_met['location'].values)
area_df = area_df.reindex(list(monthly_q['location'].values)).loc[:,0].to_xarray().values # This is the re-ordered areas according to the master key
area_m2 = (area_df * 1000000) # convert polygon areas from km^2 to m^2
monthly_q = (monthly_q / np.tile(area_m2, (len(monthly_q['time']), 1))) * 1000 # convert discharge volume (m^3) to length (m) and then to mm

# Convert to a DataArray with attributes and title
Monthly_Q = xr.DataArray(monthly_q, coords=monthly_q.coords, attrs={'standard_name': 'Monthy Discharge', 'units': 'mm'})
Monthly_Q.name = 'mo_discharge'
Monthly_Q

Create an xarray.Dataset and check the format to insure everything is correct

In [ ]:
metdata_input = xr.merge([Monthly_Temp, Monthly_Precip, Monthly_Q])
check_format(metdata_input)

Here is the final dataset that will be used with the model. 

In [ ]:
metdata_input.to_netcdf(r'C:\Users\CND905\Downloaded_Programs\mwb_flow\Examples\dataprep_Example1.1_PullData_output.nc')
metdata_input

## Examples of Code from Above Functions (for refrence only)

<ins>The following code is identical to code found in the above fuctions or of earlier approchs to pulling, calculating area weighted volumes or means of polygons. The following cells do not need to be run in this example</ins>

Below is the code used in the 'get_gridmet_for_polygons()' and 'get_gridmet_for_polygons_with_xvec()' functions. They are included for testing or troubleshooting if needed. The third example shows code similar to 'get_gridmet_at_points()' but it has been modified to pull metdata and calculate area weighted volume for precipitation is shown below. It is included as an example of earlier approches.  

In [6]:
# Give arguments for upcoming functions
in_geom = exres
gdf_index_col = "gageID"
start='2016-08-01'
end='2016-11-30'
crs = 4326

### get_gridmet_for_polygons_with_geocube() 

In [ ]:
var_list = []
for p in GRIDMET_PARAMS:
    bnds = in_geom.total_bounds
    gmet = GridMet(variable=p, start=start, end=end,
                    bbox=BBox(bnds[0] - 0.5, bnds[2] + 0.5, bnds[3] + 0.5, bnds[1] - 0.5))
    gmet = gmet.subset_nc(return_array=True)
    gmet_input = gmet[list(gmet.data_vars)[0]]

    if p == 'pr':
        gmet_input = gmet_input/1000 #convert from mm to meters
        vol_xds = gt.grid_area_weighted_volume(gmet_input, in_geom, gdf_index_col)
        # vol_xds = vol_xds.drop('area')
    else:
        # This is done for each output of the above code
        in_geom[gdf_index_col] = in_geom[gdf_index_col].astype(int) 
        gmet_input = gmet_input.rio.write_crs(input_crs=4326).rio.clip(in_geom.geometry.values, in_geom.crs)
        gmet_input.name = p

        # Added coords previously not needed in package example
        grid_out = make_geocube(vector_data=in_geom, measurements=[gdf_index_col], like=gmet_input).set_coords(gdf_index_col)

        for date in range(0, len(gmet_input.time.values)):
            gmet_ts = gmet_input[date,:,:]
            grid_ts = grid_out
            
            grid_ts[p] = (grid_out.dims, gmet_ts.values, gmet_ts.attrs, gmet_ts.encoding)
            grid_ts = grid_out.drop("spatial_ref").groupby(grid_out[gdf_index_col]).mean()

            xda = grid_ts[p]
            xda = xda.expand_dims({"time": 1}).assign_coords(time=('time', [gmet_ts.time.values]))
            var_list.append(xda)
    xds = xr.merge(var_list)

# Add lat coordinate and make match order of xds locations
lat_df = pd.DataFrame((in_geom.geometry.bounds['miny'] + in_geom.geometry.bounds['maxy']) / 2).set_index(in_geom[gdf_index_col])
lat_df = lat_df.reindex(list(xds[gdf_index_col].values.astype(int)))
    
xds = xr.Dataset(
    {
        "max_temp": (['time', 'location'], xds["tmmx"].values, {'standard_name': 'Maximum Temperature',
                                                                    'units': 'Kelvin'}),
        "min_temp": (['time', 'location'], xds["tmmn"].values, {'standard_name': 'Maximum Temperature',
                                                                    'units': 'Kelvin'})
    },
    coords={
        "lat": (['location'], list(lat_df.iloc[:,0]), {'standard_name': 'latitude',
                                        'long_name': 'location_latitude',
                                        'units': 'degrees',
                                        'crs': '4326'}),
        "location": (['location'], xds[gdf_index_col].values.astype(int), {'long_name': 'location_identifier',
                                        'cf_role': 'timeseries_id'}), # Keep the order of xds
        "time": xds['time'].values
    },
    attrs={
        "featureType": 'timeSeries',
    }
)

if 'pr' in GRIDMET_PARAMS:
    output = xr.merge([xds, vol_xds])# vol_xds reorders to match xds
else: 
    output = xds

output


### get_gridmet_for_polygons_with_xvec()

In [ ]:
# Pull gridmet data and calculate zonal stats of polygons
in_geom = in_geom.to_crs(crs)

gmet_list = []
for p in GRIDMET_PARAMS:
    bnds = in_geom.total_bounds
    gmet = GridMet(variable=p, start=start, end=end,
                    bbox=BBox(bnds[0] - 0.5, bnds[2] + 0.5, bnds[3] + 0.5, bnds[1] - 0.5))
    gmet = gmet.subset_nc(return_array=True)
    gmet_input = gmet[list(gmet.data_vars)[0]]
    if p == 'pr':
        gmet_input = gmet_input / 1000  # convert from mm to meters
        ds_sum = gmet_input.xvec.zonal_stats(geometry=in_geom.geometry, x_coords="lon", y_coords="lat", stats="sum(coverage_weight=area_spherical_m2)", method="exactextract")
        ds_sum.name = 'precipitation_amount'
    else:
        gmet_list.append(gmet_input)
ds = xr.merge(gmet_list)
ds = ds.xvec.zonal_stats(geometry=in_geom.geometry, x_coords="lon", y_coords="lat", stats="mean", method="exactextract")

ds = xr.merge([ds, ds_sum])

# Add coords
## Make and reorder the location ID coordinates
loc_df = pd.DataFrame(in_geom[gdf_index_col]).set_index(in_geom.geometry)
loc_df = loc_df.reindex(in_geom.geometry.values)

## Calculate and reorder lat coordinates
lat_df = pd.DataFrame((in_geom.geometry.bounds['miny'] + in_geom.geometry.bounds['maxy']) / 2).set_index(in_geom.geometry)
lat_df = lat_df.reindex(in_geom.geometry.values)

area_df = pd.DataFrame(in_geom.to_crs(5071).area / (1000 ** 2)).set_index(in_geom.geometry)
area_df = area_df.reindex(in_geom.geometry.values)

xds = xr.Dataset(
    {
        "max_temp": (['location', 'time'], ds['daily_maximum_temperature'].values, {'standard_name': 'Maximum Temperature',
                                                                    'units': 'Kelvin'}),
        "min_temp": (['location', 'time'], ds['daily_minimum_temperature'].values, {'standard_name': 'Maximum Temperature',
                                                                    'units': 'Kelvin'}),
        "precip_volume": (['location', 'time'], ds['precipitation_amount'].values, {'standard_name': 'Precipitation Volume',
                                                                    'units': 'm^3'})
    },
    coords={
        "lat": (['location'], list(lat_df.iloc[:,0]), {'standard_name': 'latitude',
                                        'long_name': 'location_latitude',
                                        'units': 'degrees',
                                        'crs': '4326'}),
        "area": (['location'], list(area_df.iloc[:,0]), {'standard_name': 'area',
                                                  'long_name': 'input_shape_area',
                                                  'units': 'km^2'}),
        "location": (['location'], list(loc_df.iloc[:,0]), {'long_name': 'location_identifier',
                                        'cf_role': 'timeseries_id'}),
        "time": ds['time'].values
    },
    attrs={
        "featureType": 'timeSeries',
    }
)
xds = xds.transpose('time', 'location')

xds



### Updated and archived get_gridmet_at_points()

In [ ]:
# get_gridmet_at_points()
in_geom = exres
gdf_index_col = "gageID"
start='2016-01-01'
end='2016-01-05'
crs = 4326

# overwrite configuration to run initial approch for pulling gridmet data and performing zonal stats
GRIDMET_PARAMS = ['tmmn', 'tmmx']
VOL_PARAMS = ['pr']


if gdf_index_col is not None:
    ixcol = gdf_index_col
else:
    in_geom['ixcol'] = in_geom.index
    ixcol = 'ixcol'

location_ids = in_geom[ixcol].to_list()

if (in_geom.geometry.geom_type == 'Point').all():
    coords = list(zip(in_geom.geometry.x, in_geom.geometry.y))
elif (in_geom.geometry.geom_type == 'Polygon').all():
    coords = list(zip(in_geom.geometry.centroid.x, in_geom.geometry.centroid.y))
else:
    coords = None
    raise ValueError("Mixed geometry types were found in the input GeoDataFrame. Mixed Geometry is not supported.")

loc_lat = []
loc_lon = []
loc_elev = py3dep.elevation_bycoords(coords, crs=crs)  # only 4326 or NAD83 works with py3dep

if isinstance(loc_elev, list):
    loc_elev = loc_elev
else:
    loc_elev = [loc_elev]

loc_gdf = in_geom[['{0}'.format(ixcol), 'geometry']]

print("Retrieving GridMET cells...")
gmt_cells = get_gridmet_cells(loc_gdf)
unq_cells = gmt_cells['cell_id'].unique()
print("{0} unique GridMET cells found for {1} input features.".format(len(unq_cells), len(loc_gdf[ixcol])))

gmt_cntrs = gmt_cells.drop_duplicates(subset='cell_id').centroid

# Parameters retrieved to be averaged over watershed area here
tmmn = []
tmmx = []

cdsets = {}
print("Fetching GridMET data for unique cells...")
for cell in tqdm(unq_cells, desc='Cells'):
    clon = gmt_cntrs[cell].x
    clat = gmt_cntrs[cell].y
    datasets = []
    for p in GRIDMET_PARAMS:
        s = start
        e = end
        ds = GridMet(p, start=s, end=e, lat=clat, lon=clon).get_point_timeseries()
        datasets.append(ds)
    cdsets[cell] = datasets

# Parameters retried to be converted wot weighted volumes here
if len(VOL_PARAMS)> 1:
    raise ValueError("GRIDtools.grid_area_weighted_volume() is only compatible with the precip parameter")

# volparam_list = []
for p in VOL_PARAMS:
    bnds = in_geom.total_bounds
    gmet = GridMet(variable= p, start=start, end=end, bbox=BBox(bnds[0]-0.5, bnds[2]+0.5, bnds[3]+0.5, bnds[1]-0.5))

    gmet = gmet.subset_nc(return_array=True)
    gmet_input = gmet[list(gmet.data_vars)[0]]
    vol_xds = gt.grid_area_weighted_volume(gmet_input, in_geom, 'gageID')
    # volparam_list.append(vol_xds)
# xr.merge(volparam_list)

for i in range(len(coords)):
    c = coords[i]
    loc = location_ids[i]
    gmtcell_ids = gmt_cells[gmt_cells[ixcol] == loc]
    lon, lat = c
    loc_lat.append(lat)
    loc_lon.append(lon)


    if len(gmtcell_ids.index) > 1:

        tmmnm = []
        tmmxm = []

        for cid in gmtcell_ids['cell_id']:
            dset = cdsets[cid]

            tmmnm.append(dset[GRIDMET_PARAMS.index('tmmn')])
            tmmxm.append(dset[GRIDMET_PARAMS.index('tmmx')])

        tmmnm_d = pd.concat(tmmnm)
        tmmxm_d = pd.concat(tmmxm)

        tmmn.append(tmmnm_d.groupby(tmmnm_d.index).mean())
        tmmx.append(tmmxm_d.groupby(tmmxm_d.index).mean())

    else:
        dset = cdsets[gmtcell_ids['cell_id'].values[0]]
        tmmn.append(dset[GRIDMET_PARAMS.index('tmmn')])
        tmmx.append(dset[GRIDMET_PARAMS.index('tmmx')])

mean_xds = xr.Dataset(
    {
        "min_temp": (['time', 'location'], pd.concat(tmmn, axis=1), {'standard_name': 'Minimum Temperature',
                                                                    'units': 'Kelvin'}),
        "max_temp": (['time', 'location'], pd.concat(tmmx, axis=1), {'standard_name': 'Maximum Temperature',
                                                                    'units': 'Kelvin'})
    },
    coords={
        "lat": (['location'], loc_lat, {'standard_name': 'latitude',
                                        'long_name': 'location_latitude',
                                        'units': 'degrees',
                                        'crs': '4326'}),
        "lon": (['location'], loc_lon, {'standard_name': 'longitude',
                                        'long_name': 'location_longitude',
                                        'units': 'degrees',
                                        'crs': '4326'}),
        "elev": (['location'], loc_elev, {'standard_name': 'elevation',
                                        'long_name': 'location_elevation',
                                        'units': 'meters'}),
        "location": (['location'], location_ids, {'long_name': 'location_identifier',
                                        'cf_role': 'timeseries_id'}),
        "time": tmmn[0].index
    },
    attrs={
        "featureType": 'timeSeries',
    }
)

xr.merge([mean_xds, vol_xds])